<a href="https://colab.research.google.com/github/JavalVyas2000/Estimation-of-RUL/blob/main/Estimation_of_RUL_using_ARIMA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# installing all the prerequisited for connecting with the drive.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Installing ARIMA 
!pip3 install pmdarima

In [ ]:
# Installing the python libraries that we may use
from keras.preprocessing.sequence import TimeseriesGenerator

import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
# from datetime import datetime, date, timedelta

import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
%matplotlib inline

# statistics
from statsmodels.distributions.empirical_distribution import ECDF

# time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm


import warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.arima_model import ARIMA
import statsmodels.tools.eval_measures as  em
from sklearn.metrics import  mean_squared_error
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from IPython.display import display
from pylab import rcParams
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn import linear_model


from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from datetime import date

In [ ]:
# installing openpyxl to adjust to the version
!pip install openpyxl==3.0.0

In [ ]:
# reading the data
data1=pd.read_excel('gdrive/MyDrive/RUL data/Discap_Calculated.xlsx')
valide=pd.read_excel('gdrive/MyDrive/RUL data/Discap_Calculated.xlsx')

In [ ]:
# cleaning the data and making the cycles continuous

data1['Cycle']=data1['Cycle']//2
cycle_no=[]
p=data1['Cycle'][data1.shape[0]-1]
for i in range(p):
  cycle_no.append(i+1)
Discap=[None]*len(cycle_no)
for i in range(data1.shape[0]-1):
  Discap[data1['Cycle'][i]]=data1['Discap'][i]

In [ ]:
# creating a dataframe and interpolating the missing values
df1 = pd.DataFrame(Discap,cycle_no)
df1.interpolate(method ='linear', limit_direction ='forward',axis=0,inplace=True)
df1.dropna(inplace=True)

In [ ]:
# Creating a new dataframe that is differenced once. 
df1_dif=df1.diff(1)
df1_dif.dropna(inplace=True)
plt.plot(df1_dif)

In [ ]:
# dividing the dataframe into train and test modes. 
size=72  # Size of training data set is either 72 or 120 based on the https://www.mdpi.com/1996-1073/14/21/7206 research paper. 
df11_dif=df1_dif[0:size] 
test1= df1_dif[size:]
df11_dif.plot(figsize=(12,5))

In [ ]:
import pmdarima as pm
from pmdarima.arima import auto_arima
import statsmodels.tsa.arima.model as arima

In [ ]:
# build the arima model based on the parameters found analytically
model = arima.ARIMA(df11_dif, order = (1, 0, 60)) 

result = model.fit() 
result.summary() 

# start the predictions from the last cycle of the training dataset to the end of test
start_index = len(df11_dif)
end_index = len(df1)
prediction = result.predict(start_index, end_index) 
print(prediction)

# plot predictions and actual values 
plt.plot(prediction,label='Prediction')
plt.plot(test1,label='Test')
plt.legend(loc='best')

In [ ]:
# create a function to get the original values from the differenced values
def inverse_difference(last_ob, value):
	return value + last_ob

# take the inverse of the differenced values to get the original predictions and compare it with the test
predictions = [inverse_difference(df1.values[size+i], prediction.values[i]) for i in range(len(prediction)-1)]
test1_inv=[inverse_difference(df1.values[size+i], test1.values[i]) for i in range(len(test1)-1)]

# plot the results of the predictions and test
plt.figure(figsize=(10,5))
plt.plot(np.arange(size,size+len(predictions)),predictions,label='Prediction')
plt.plot(np.arange(size,size+len(test1_inv)),test1_inv,label='Test')
plt.legend(loc='best')
plt.xlabel('Cycle_no')
plt.ylabel('Discap')
plt.title('B0006')

In [ ]:
# evaluation metrics - RMSE, MSE, MAPE, MAE
import math
testScore1 = math.sqrt(mean_squared_error(test1_inv, predictions[:-2]))
print('Test Score1: %.4f RMSE' % (testScore1))
testScore11 = (mean_squared_error(test1_inv, predictions[:-2]))
print('Test Score11: %.4f MSE' % (testScore11))
pred=np.array(predictions[:-2])
actual=np.array(test1_inv)
MAPE1=np.mean(np.abs((actual-pred)/actual))*100
print('Test Score: %.4f MAPE' % (MAPE1))
MAE1=np.mean(np.abs((actual-pred)))
print('Test Score: %.4f MAE' % (MAE1))